In [1]:
import pyspark
sc = pyspark.SparkContext('local[*]')
rdd = sc.parallelize(range(1000))
rdd.takeSample(False, 5)

[754, 329, 685, 893, 371]

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

In [3]:
df1 = pd.read_csv("test_set.csv")
df2 = pd.read_csv("training_set_labels.csv")
df3 = pd.read_csv("training_set_values.csv")
df = df3

In [4]:
df1

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,2013-05-03,Germany Republi,1210,CES,37.169807,-3.253847,Area Three Namba 27,0,...,per bucket,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
59396,27263,4700.0,2011-05-07,Cefa-njombe,1212,Cefa,35.249991,-9.070629,Kwa Yahona Kuvala,0,...,annually,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
59397,37057,0.0,2011-04-11,NaN,0,NaN,34.017087,-8.750434,Mashine,0,...,monthly,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
59398,31282,0.0,2011-03-08,Malec,0,Musa,35.861315,-6.378573,Mshoro,0,...,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump


In [5]:
df2

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional
...,...,...
59395,60739,functional
59396,27263,functional
59397,37057,functional
59398,31282,functional


In [6]:
df3.shape

(14850, 40)

In [7]:
df1.wpt_name.isin(df.wpt_name)
df.wpt_name.nunique()

10840

In [8]:
df.info()
df = df.dropna(thresh = 5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14850 entries, 0 to 14849
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     14850 non-null  int64  
 1   amount_tsh             14850 non-null  float64
 2   date_recorded          14850 non-null  object 
 3   funder                 13981 non-null  object 
 4   gps_height             14850 non-null  int64  
 5   installer              13973 non-null  object 
 6   longitude              14850 non-null  float64
 7   latitude               14850 non-null  float64
 8   wpt_name               14850 non-null  object 
 9   num_private            14850 non-null  int64  
 10  basin                  14850 non-null  object 
 11  subvillage             14751 non-null  object 
 12  region                 14850 non-null  object 
 13  region_code            14850 non-null  int64  
 14  district_code          14850 non-null  int64  
 15  lg

In [9]:
df.iloc[:, 0:10]

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0
...,...,...,...,...,...,...,...,...,...,...
14845,39307,0.0,2011-02-24,Danida,34,Da,38.852669,-6.582841,Kwambwezi,0
14846,18990,1000.0,2011-03-21,Hiap,0,HIAP,37.451633,-5.350428,Bonde La Mkondoa,0
14847,28749,0.0,2013-03-04,NaN,1476,NaN,34.739804,-4.585587,Bwawani,0
14848,33492,0.0,2013-02-18,Germany,998,DWE,35.432732,-10.584159,Kwa John,0


In [10]:
df.amount_tsh.value_counts()

0.0         10410
500.0         772
50.0          631
1000.0        370
20.0          349
            ...  
200000.0        1
100000.0        1
70000.0         1
2550.0          1
7000.0          1
Name: amount_tsh, Length: 68, dtype: int64

In [11]:
df["tsh_0"] = (df["amount_tsh"] == 0).astype(int)
df

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,tsh_0
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other,1
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,...,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe,1
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other,1
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,...,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other,1
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14845,39307,0.0,2011-02-24,Danida,34,Da,38.852669,-6.582841,Kwambwezi,0,...,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe,1
14846,18990,1000.0,2011-03-21,Hiap,0,HIAP,37.451633,-5.350428,Bonde La Mkondoa,0,...,salty,salty,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,0
14847,28749,0.0,2013-03-04,NaN,1476,NaN,34.739804,-4.585587,Bwawani,0,...,soft,good,insufficient,insufficient,dam,dam,surface,communal standpipe,communal standpipe,1
14848,33492,0.0,2013-02-18,Germany,998,DWE,35.432732,-10.584159,Kwa John,0,...,soft,good,insufficient,insufficient,river,river/lake,surface,communal standpipe,communal standpipe,1


In [12]:
df.funder.value_counts()
df["gov_funding"] = (df.funder == "Government Of Tanzania").astype(int)
df

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,tsh_0,gov_funding
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,...,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other,1,0
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,...,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe,1,1
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,...,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other,1,0
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,...,good,dry,dry,shallow well,shallow well,groundwater,other,other,1,0
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,...,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14845,39307,0.0,2011-02-24,Danida,34,Da,38.852669,-6.582841,Kwambwezi,0,...,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe,1,0
14846,18990,1000.0,2011-03-21,Hiap,0,HIAP,37.451633,-5.350428,Bonde La Mkondoa,0,...,salty,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,0,0
14847,28749,0.0,2013-03-04,NaN,1476,NaN,34.739804,-4.585587,Bwawani,0,...,good,insufficient,insufficient,dam,dam,surface,communal standpipe,communal standpipe,1,0
14848,33492,0.0,2013-02-18,Germany,998,DWE,35.432732,-10.584159,Kwa John,0,...,good,insufficient,insufficient,river,river/lake,surface,communal standpipe,communal standpipe,1,0


In [13]:
df.installer.value_counts()

DWE                   4349
Government             457
RWE                    292
Commu                  287
DANIDA                 255
                      ... 
Wanjoda                  1
D                        1
VE                       1
Private Technician       1
VITECOS                  1
Name: installer, Length: 1091, dtype: int64

In [14]:
df.wpt_name.value_counts()

none                 877
Shuleni              435
Zahanati             204
Msikitini            112
Sokoni                68
                    ... 
Kwa Masudy Rajabu      1
Kwa Hamis Rajab        1
Kwa Hungilo            1
Maji                   1
Kwasesitu              1
Name: wpt_name, Length: 10840, dtype: int64

In [15]:
df.num_private.value_counts()

0      14656
1         21
5         14
8         12
15        12
6         11
3         11
4         10
93         9
102        8
2          8
17         7
11         7
47         6
16         5
22         5
65         5
32         5
7          5
45         5
39         4
34         3
50         2
120        2
80         2
12         2
41         2
25         2
180        2
23         1
13         1
420        1
10         1
26         1
141        1
669        1
Name: num_private, dtype: int64

In [16]:
df.iloc[:, 10:20]

,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by
0,Internal,Magoma,Manyara,21,3,Mbulu,Bashay,321,True,GeoData Consultants Ltd
1,Pangani,Kimnyak,Arusha,2,2,Arusha Rural,Kimnyaki,300,True,GeoData Consultants Ltd
2,Internal,Msatu,Singida,13,2,Singida Rural,Puma,500,True,GeoData Consultants Ltd
3,Ruvuma / Southern Coast,Kipindimbi,Lindi,80,43,Liwale,Mkutano,250,NaN,GeoData Consultants Ltd
4,Ruvuma / Southern Coast,Losonga,Ruvuma,10,3,Mbinga,Mbinga Urban,60,NaN,GeoData Consultants Ltd
...,...,...,...,...,...,...,...,...,...,...
14845,Wami / Ruvu,Yombo,Pwani,6,1,Bagamoyo,Yombo,20,True,GeoData Consultants Ltd
14846,Pangani,Mkondoa,Tanga,4,7,Kilindi,Mvungwe,2960,True,GeoData Consultants Ltd
14847,Internal,Juhudi,Singida,13,2,Singida Rural,Ughandi,200,True,GeoData Consultants Ltd
14848,Lake Nyasa,Namakinga B,Ruvuma,10,2,Songea Rural,Maposeni,150,True,GeoData Consultants Ltd


In [64]:
df.basin.value_counts()
#df.basin.isna().sum()
#final = pd.DataFrame()
#final["basin"] = df.basin
#final["tsh_0"] = df.tsh_0

Lake Victoria              2623
Pangani                    2203
Rufiji                     2011
Internal                   1857
Lake Tanganyika            1620
Wami / Ruvu                1590
Lake Nyasa                 1247
Ruvuma / Southern Coast    1094
Lake Rukwa                  605
Name: basin, dtype: int64

In [18]:
df.subvillage.value_counts()

Shuleni         140
Majengo         129
Madukani        121
Kati             94
Sokoni           62
               ... 
Makasa            1
Sangamkuu         1
Kifumbi           1
Mbelekese         1
Lukama Chini      1
Name: subvillage, Length: 8443, dtype: int64

In [66]:
len(df.region.value_counts())
#final["region"] = df.region

21

In [20]:
df.region_code.value_counts()

17    1323
11    1308
12    1120
3     1115
5     1039
18     859
19     777
16     717
2      685
10     666
4      632
1      578
14     509
20     482
13     443
15     434
6      423
21     389
80     298
60     273
90     216
7      215
9      109
99      89
24      76
8       75
Name: region_code, dtype: int64

In [85]:
len(df.district_code.value_counts())
#final["district_code"] = df.district_code
#final

0         3
1         2
2         2
3        43
4         3
         ..
14845     1
14846     7
14847     2
14848     2
14849     3
Name: district_code, Length: 14850, dtype: object

In [22]:
df.lga.value_counts()

Njombe          625
Moshi Rural     315
Bariadi         308
Rungwe          275
Kasulu          275
               ... 
Songea Urban     23
Tanga            22
Arusha Urban     19
Lindi Urban       3
Nyamagana         1
Name: lga, Length: 125, dtype: int64

In [23]:
df.ward.value_counts()

Igosi        79
Imalinyi     66
Siha Kati    65
Mdandu       61
Kitunda      57
             ..
Bassotu       1
Gombero       1
Msangeni      1
Nyamidaho     1
Santilya      1
Name: ward, Length: 1959, dtype: int64

In [24]:
df.population.value_counts()
final["population"] = df.population

In [25]:
df.public_meeting.value_counts()
#df.public_meeting.isna().sum()


True     12738
False     1291
Name: public_meeting, dtype: int64

In [26]:
df.public_meeting = df.public_meeting.fillna(False)
final["public_meeting"] = df.public_meeting.astype(int)
final.isna().sum()

basin             0
region            0
district_code     0
population        0
public_meeting    0
dtype: int64

In [27]:
df.iloc[:, 20:30]

,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment
0,Parastatal,NaN,True,2012,other,other,other,parastatal,parastatal,never pay
1,VWC,TPRI pipe line,True,2000,gravity,gravity,gravity,vwc,user-group,never pay
2,VWC,P,NaN,2010,other,other,other,vwc,user-group,never pay
3,VWC,NaN,True,1987,other,other,other,vwc,user-group,unknown
4,Water Board,BRUDER,True,2000,gravity,gravity,gravity,water board,user-group,pay monthly
...,...,...,...,...,...,...,...,...,...,...
14845,VWC,Bagamoyo wate,True,1988,mono,mono,motorpump,vwc,user-group,never pay
14846,VWC,NaN,False,1994,nira/tanira,nira/tanira,handpump,vwc,user-group,pay annually
14847,VWC,NaN,NaN,2010,gravity,gravity,gravity,vwc,user-group,never pay
14848,VWC,Mradi wa maji wa maposeni,True,2009,gravity,gravity,gravity,vwc,user-group,never pay


In [86]:
df.scheme_management.value_counts()
#final["scheme_management"] = df.scheme_management

0         Parastatal
1                VWC
2                VWC
3                VWC
4        Water Board
            ...     
14845            VWC
14846            VWC
14847            VWC
14848            VWC
14849            VWC
Name: scheme_management, Length: 14850, dtype: object

In [71]:
df.permit.value_counts()
df.permit.isna().sum()

0

In [30]:
df.permit = df.permit.fillna(False).astype(int)
final["permit"] = df.permit

In [31]:
df.construction_year
final["construction_year"] = df.construction_year

In [87]:
df.extraction_type.value_counts()
#final["extraction_type"] = df.extraction_type

0              other
1            gravity
2              other
3              other
4            gravity
            ...     
14845           mono
14846    nira/tanira
14847        gravity
14848        gravity
14849        gravity
Name: extraction_type, Length: 14850, dtype: object

In [33]:
df.extraction_type_group.value_counts()

gravity            6483
nira/tanira        2051
other              1672
submersible        1593
swn 80              918
mono                763
india mark ii       629
afridev             438
rope pump           121
other handpump       83
india mark iii       37
wind-powered         35
other motorpump      27
Name: extraction_type_group, dtype: int64

In [74]:
df.extraction_type_class.value_counts()
final["extraction_type_class"] = df.extraction_type

In [75]:
df.management.value_counts()
#final["management"] = df.management

vwc                 10117
wug                  1593
water board           755
wua                   583
private operator      533
parastatal            461
other                 239
water authority       219
company               174
unknown               122
other - school         27
trust                  27
Name: management, dtype: int64

In [76]:
df.management_group.value_counts()
#final["management_group"] = df.management_group

user-group    13048
commercial      953
parastatal      461
other           266
unknown         122
Name: management_group, dtype: int64

In [37]:
df.payment.value_counts()

never pay                6364
pay per bucket           2281
pay monthly              2097
unknown                  1992
pay annually              928
pay when scheme fails     928
other                     260
Name: payment, dtype: int64

In [38]:
df.payment.value_counts()
final["payment"] = df.payment
final

,basin,region,district_code,population,public_meeting,scheme_management,permit,construction_year,extraction_type,extraction_type_class,management,management_group,payment
0,Internal,Manyara,3,321,1,Parastatal,1,2012,other,other,parastatal,parastatal,never pay
1,Pangani,Arusha,2,300,1,VWC,1,2000,gravity,gravity,vwc,user-group,never pay
2,Internal,Singida,2,500,1,VWC,0,2010,other,other,vwc,user-group,never pay
3,Ruvuma / Southern Coast,Lindi,43,250,0,VWC,1,1987,other,other,vwc,user-group,unknown
4,Ruvuma / Southern Coast,Ruvuma,3,60,0,Water Board,1,2000,gravity,gravity,water board,user-group,pay monthly
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14845,Wami / Ruvu,Pwani,1,20,1,VWC,1,1988,mono,mono,vwc,user-group,never pay
14846,Pangani,Tanga,7,2960,1,VWC,0,1994,nira/tanira,nira/tanira,vwc,user-group,pay annually
14847,Internal,Singida,2,200,1,VWC,0,2010,gravity,gravity,vwc,user-group,never pay
14848,Lake Nyasa,Ruvuma,2,150,1,VWC,1,2009,gravity,gravity,vwc,user-group,never pay


In [39]:
df.iloc[:, 30:40]

,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
...,...,...,...,...,...,...,...,...,...,...
14845,never pay,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
14846,annually,salty,salty,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
14847,never pay,soft,good,insufficient,insufficient,dam,dam,surface,communal standpipe,communal standpipe
14848,never pay,soft,good,insufficient,insufficient,river,river/lake,surface,communal standpipe,communal standpipe


In [77]:
df.water_quality.value_counts()
#final["water_quality"] = df.water_quality

soft                  12687
salty                  1226
unknown                 469
milky                   201
coloured                133
salty abandoned          84
fluoride                 44
fluoride abandoned        6
Name: water_quality, dtype: int64

In [41]:
df.quality_group.value_counts()

good        12687
salty        1310
unknown       469
milky         201
colored       133
fluoride       50
Name: quality_group, dtype: int64

In [78]:
df.quantity.value_counts()
#final["quantity"] = df.quantity

enough          8336
insufficient    3767
dry             1536
seasonal        1025
unknown          186
Name: quantity, dtype: int64

In [79]:
df.source.value_counts()
#final["source"] = df.source

shallow well            4316
spring                  4195
machine dbh             2747
river                   2352
rainwater harvesting     568
hand dtw                 234
lake                     185
dam                      184
other                     49
unknown                   20
Name: source, dtype: int64

In [49]:
#final["target"] = df.status_group
#target = final.target
final

,basin,region,district_code,population,public_meeting,scheme_management,permit,construction_year,extraction_type,extraction_type_class,management,management_group,payment,water_quality,quantity,source
0,Internal,Manyara,3,321,1,Parastatal,1,2012,other,other,parastatal,parastatal,never pay,soft,seasonal,rainwater harvesting
1,Pangani,Arusha,2,300,1,VWC,1,2000,gravity,gravity,vwc,user-group,never pay,soft,insufficient,spring
2,Internal,Singida,2,500,1,VWC,0,2010,other,other,vwc,user-group,never pay,soft,insufficient,rainwater harvesting
3,Ruvuma / Southern Coast,Lindi,43,250,0,VWC,1,1987,other,other,vwc,user-group,unknown,soft,dry,shallow well
4,Ruvuma / Southern Coast,Ruvuma,3,60,0,Water Board,1,2000,gravity,gravity,water board,user-group,pay monthly,soft,enough,spring
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14845,Wami / Ruvu,Pwani,1,20,1,VWC,1,1988,mono,mono,vwc,user-group,never pay,soft,enough,river
14846,Pangani,Tanga,7,2960,1,VWC,0,1994,nira/tanira,nira/tanira,vwc,user-group,pay annually,salty,insufficient,shallow well
14847,Internal,Singida,2,200,1,VWC,0,2010,gravity,gravity,vwc,user-group,never pay,soft,insufficient,dam
14848,Lake Nyasa,Ruvuma,2,150,1,VWC,1,2009,gravity,gravity,vwc,user-group,never pay,soft,insufficient,river


In [50]:
final

,basin,region,district_code,population,public_meeting,scheme_management,permit,construction_year,extraction_type,extraction_type_class,management,management_group,payment,water_quality,quantity,source
0,Internal,Manyara,3,321,1,Parastatal,1,2012,other,other,parastatal,parastatal,never pay,soft,seasonal,rainwater harvesting
1,Pangani,Arusha,2,300,1,VWC,1,2000,gravity,gravity,vwc,user-group,never pay,soft,insufficient,spring
2,Internal,Singida,2,500,1,VWC,0,2010,other,other,vwc,user-group,never pay,soft,insufficient,rainwater harvesting
3,Ruvuma / Southern Coast,Lindi,43,250,0,VWC,1,1987,other,other,vwc,user-group,unknown,soft,dry,shallow well
4,Ruvuma / Southern Coast,Ruvuma,3,60,0,Water Board,1,2000,gravity,gravity,water board,user-group,pay monthly,soft,enough,spring
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14845,Wami / Ruvu,Pwani,1,20,1,VWC,1,1988,mono,mono,vwc,user-group,never pay,soft,enough,river
14846,Pangani,Tanga,7,2960,1,VWC,0,1994,nira/tanira,nira/tanira,vwc,user-group,pay annually,salty,insufficient,shallow well
14847,Internal,Singida,2,200,1,VWC,0,2010,gravity,gravity,vwc,user-group,never pay,soft,insufficient,dam
14848,Lake Nyasa,Ruvuma,2,150,1,VWC,1,2009,gravity,gravity,vwc,user-group,never pay,soft,insufficient,river


In [80]:
df.scheme_management.value_counts()
#final.scheme_management = final.scheme_management.fillna("Other")

VWC                 9124
WUG                 1290
Water authority      822
Water Board          714
WUA                  668
Parastatal           444
Company              280
Private operator     263
Other                230
SWC                   26
Trust                 20
Name: scheme_management, dtype: int64

In [60]:
final.isna().sum()
final.district_code = final.district_code.astype(str)
final

,basin,region,district_code,population,public_meeting,scheme_management,permit,construction_year,extraction_type,extraction_type_class,management,management_group,payment,water_quality,quantity,source
0,Internal,Manyara,3,321,1,Parastatal,1,2012,other,other,parastatal,parastatal,never pay,soft,seasonal,rainwater harvesting
1,Pangani,Arusha,2,300,1,VWC,1,2000,gravity,gravity,vwc,user-group,never pay,soft,insufficient,spring
2,Internal,Singida,2,500,1,VWC,0,2010,other,other,vwc,user-group,never pay,soft,insufficient,rainwater harvesting
3,Ruvuma / Southern Coast,Lindi,43,250,0,VWC,1,1987,other,other,vwc,user-group,unknown,soft,dry,shallow well
4,Ruvuma / Southern Coast,Ruvuma,3,60,0,Water Board,1,2000,gravity,gravity,water board,user-group,pay monthly,soft,enough,spring
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14845,Wami / Ruvu,Pwani,1,20,1,VWC,1,1988,mono,mono,vwc,user-group,never pay,soft,enough,river
14846,Pangani,Tanga,7,2960,1,VWC,0,1994,nira/tanira,nira/tanira,vwc,user-group,pay annually,salty,insufficient,shallow well
14847,Internal,Singida,2,200,1,VWC,0,2010,gravity,gravity,vwc,user-group,never pay,soft,insufficient,dam
14848,Lake Nyasa,Ruvuma,2,150,1,VWC,1,2009,gravity,gravity,vwc,user-group,never pay,soft,insufficient,river


In [61]:
final2 = pd.get_dummies(final)

In [63]:
final2

,population,public_meeting,permit,construction_year,basin_Internal,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,basin_Lake Victoria,basin_Pangani,...,source_dam,source_hand dtw,source_lake,source_machine dbh,source_other,source_rainwater harvesting,source_river,source_shallow well,source_spring,source_unknown
0,321,1,1,2012,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,300,1,1,2000,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
2,500,1,0,2010,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,250,0,1,1987,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,60,0,1,2000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14845,20,1,1,1988,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
14846,2960,1,0,1994,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
14847,200,1,0,2010,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
14848,150,1,1,2009,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


# Models

In [84]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
X_train, X_test, y_train, y_test = train_test_split(
    final2, target, test_size=0.25, random_state=13)

NameError: name 'target' is not defined

In [54]:
clf = AdaBoostClassifier()
clf.fit(X_train, y_train)

AdaBoostClassifier()

In [55]:
clf.score(X_test, y_test)

0.722020202020202

In [58]:
clf = AdaBoostClassifier()
param_grid = {
    "n_estimators" : [100,200],
    "learning_rate" : [.001, .01, .1, .2, .5]
}

ada_grid_search = GridSearchCV(clf, param_grid, cv=3, return_train_score=True)

ada_grid_search.fit(X_train, y_train)


#training_preds = clf.predict(X_train)
#val_preds = clf.predict(X_test)
#training_accuracy = accuracy_score(y_train, training_preds)
#val_accuracy = accuracy_score(y_test, val_preds)

GridSearchCV(cv=3, estimator=AdaBoostClassifier(),
             param_grid={'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.5],
                         'n_estimators': [100, 200]},
             return_train_score=True)

In [60]:
ada_grid_search.score(X_test, y_test)

0.7321212121212122

In [61]:
from sklearn.preprocessing import StandardScaler
gbt_clf = GradientBoostingClassifier()
gbt_clf.fit(X_train, y_train)
gbt_clf.score(X_test,y_test)

0.7525925925925926

In [62]:
gbt_clf.score(X_train, y_train)

0.7490909090909091

In [63]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier()

dt_param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 2, 3, 4, 5, 6],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 3, 4, 5, 6]
}
dt_grid_search = GridSearchCV(dt_clf, dt_param_grid, cv=3, return_train_score=True)
dt_grid_search.fit(X_train, y_train)

dt_gs_training_score = np.mean(dt_grid_search.cv_results_['mean_train_score'])

In [64]:
dt_grid_search.best_score_

0.7571492704826038

In [65]:
dt_gs_testing_score = np.mean(dt_grid_search.cv_results_['mean_test_score'])
dt_gs_testing_score

0.7034502431724654

In [66]:
dt_grid_search.best_score_

0.7571492704826038

In [81]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
forest = RandomForestClassifier()
rf_param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 2, 6, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 3 ,6]
}

rf_grid_search = GridSearchCV(forest, rf_param_grid, cv=3)
rf_grid_search.fit(X_train, y_train)

NameError: name 'GridSearchCV' is not defined

In [82]:
rf_grid_search.best_params_
#rf_grid_search.score(X_test, y_test)

{'criterion': 'entropy',
 'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 10}

In [59]:
final2.to_csv("predict.csv", index=False)

In [86]:
predict = rf_grid_search.predict(X_train_transformed)
for i in predict:
    print(i)

non functional
non functional
non functional
non functional
functional
functional
functional
functional
functional
non functional
non functional
non functional
functional
non functional
non functional
functional
functional needs repair
non functional
non functional
functional
functional
functional
functional
non functional
non functional
functional
non functional
non functional
functional
functional
functional
non functional
functional
non functional
non functional
non functional
non functional
functional
non functional
functional
functional
non functional
functional
functional
non functional
functional
functional
non functional
non functional
non functional
non functional
functional
non functional
non functional
functional
non functional
non functional
functional
non functional
non functional
non functional
functional
non functional
non functional
functional
functional
non functional
non functional
functional
non functional
functional
functional
non functional
non functional
functiona

functional
non functional
non functional
non functional
non functional
functional
functional
functional
functional
functional
functional
non functional
functional
functional needs repair
non functional
non functional
functional
non functional
non functional
non functional
functional
functional
functional
functional
functional
non functional
functional
functional
functional
functional
non functional
non functional
non functional
functional
functional
functional
functional
non functional
functional
functional
functional
non functional
non functional
functional
functional
functional
functional
functional
functional
non functional
functional
non functional
functional
functional
functional
functional
functional
non functional
non functional
non functional
functional
functional
functional
non functional
functional
functional
functional
non functional
non functional
functional
non functional
non functional
functional
non functional
non functional
functional
non functional
functional
functiona

functional
functional
non functional
functional
non functional
functional
non functional
non functional
non functional
functional
functional
functional
functional needs repair
functional
functional
functional
non functional
non functional
functional
non functional
functional
functional
non functional
functional
functional
functional
functional
functional
functional
functional
functional
functional
functional
non functional
functional needs repair
non functional
non functional
functional
non functional
functional
non functional
non functional
functional
functional
functional
non functional
non functional
functional
functional
functional
non functional
non functional
functional
functional
functional
functional
non functional
non functional
functional
functional
non functional
functional
non functional
non functional
non functional
non functional
non functional
non functional
non functional
non functional
non functional
non functional
functional
non functional
non functional
functional
fu

non functional
functional
non functional
functional
non functional
non functional
non functional
non functional
functional
functional
non functional
functional
functional
functional
functional
functional
non functional
non functional
functional
non functional
functional
functional
non functional
non functional
functional
non functional
functional
functional
non functional
non functional
non functional
non functional
non functional
non functional
functional
functional
functional
functional
functional
non functional
functional needs repair
non functional
functional
functional
functional
functional
functional
non functional
non functional
functional
non functional
functional
non functional
functional
functional
functional
non functional
non functional
non functional
functional
functional
functional
functional
non functional
functional
functional
functional
functional
non functional
functional
functional
non functional
functional
functional
non functional
functional
non functional
function

non functional
functional
non functional
functional
functional
non functional
functional
functional
functional needs repair
functional
functional
non functional
functional
non functional
functional
non functional
non functional
functional
functional
functional
functional
non functional
non functional
functional
non functional
functional
functional
functional
functional
functional
functional
functional
non functional
non functional
functional
non functional
functional
non functional
functional
functional
non functional
functional
functional
non functional
functional
non functional
functional
functional
non functional
non functional
non functional
functional
functional
non functional
functional
functional
functional
non functional
non functional
non functional
functional
functional
functional
non functional
non functional
functional
non functional
functional
non functional
functional
functional
functional
functional
non functional
non functional
functional
functional
non functional
funct

...

functional
functional
functional
non functional
functional
functional
functional
functional
functional
non functional
non functional
non functional
non functional
non functional
non functional
functional
non functional
non functional
functional
non functional
non functional
non functional
non functional
functional
non functional
non functional
non functional
functional
non functional
non functional
non functional
functional
functional
non functional
functional
non functional
functional
functional
non functional
non functional
functional
non functional
non functional
functional
functional needs repair
functional
non functional
functional
non functional
non functional
functional
non functional
functional
non functional
functional
non functional
non functional
functional
functional
non functional
functional
non functional
functional
non functional
non functional
functional
non functional
non functional
functional
functional needs repair
functional
non functional
functional
functional
non 

non functional
non functional
non functional
functional
non functional
non functional
functional
functional
functional
functional
functional
functional needs repair
non functional
functional
non functional
non functional
functional
functional
functional
functional needs repair
non functional
functional
non functional
functional
non functional
non functional
functional
functional
functional
functional
non functional
functional
non functional
non functional
non functional
non functional
functional
functional
functional
functional
functional needs repair
non functional
non functional
non functional
functional
functional
functional
non functional
functional
functional
non functional
non functional
functional
non functional
functional
functional
functional
non functional
non functional
non functional
functional
functional
functional
non functional
non functional
functional
functional
non functional
functional
non functional
functional
non functional
non functional
non functional
non functio

non functional
non functional
functional
non functional
functional
non functional
functional
non functional
functional
non functional
non functional
functional
functional
non functional
non functional
functional
non functional
non functional
non functional
functional
functional
functional
functional
non functional
non functional
functional
non functional
non functional
functional
non functional
non functional
functional
functional
non functional
non functional
non functional
non functional
non functional
non functional
functional
non functional
non functional
functional
non functional
functional
functional
functional
non functional
functional
non functional
non functional
functional
functional
non functional
non functional
functional
functional
functional
non functional
non functional
functional
functional
functional
functional
non functional
functional
non functional
non functional
non functional
non functional
non functional
functional
non functional
non functional
non functional
non

functional
non functional
non functional
functional
non functional
non functional
functional
non functional
non functional
non functional
functional
functional
functional
functional
functional
functional
non functional
functional
functional
non functional
functional
non functional
functional
functional
functional
non functional
functional
non functional
functional
functional
functional
functional
functional
non functional
functional
functional
functional
non functional
functional
functional
functional
functional
non functional
functional
functional
non functional
functional
functional
non functional
functional
functional
non functional
functional
non functional
non functional
functional
functional
non functional
non functional
non functional
non functional
functional
functional
non functional
functional
functional
functional
functional
functional
non functional
non functional
functional
non functional
non functional
non functional
functional
functional
non functional
functional
functio

non functional
non functional
functional
non functional
non functional
functional
functional
non functional
functional
functional
functional
functional
non functional
functional
functional
functional
functional
functional
functional
functional
non functional
non functional
functional
non functional
non functional
functional
functional
functional
non functional
functional
functional
functional
functional
non functional
functional
non functional
functional
non functional
functional
non functional
functional
non functional
functional
functional
non functional
non functional
functional
functional
non functional
functional
functional
functional
functional
functional
functional
non functional
functional
functional
non functional
functional
functional
non functional
non functional
functional
functional needs repair
functional
functional
functional
non functional
functional
non functional
functional
non functional
functional
non functional
functional
functional
functional
functional
functional

In [88]:
df3

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,...,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,...,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,...,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14845,39307,0.0,2011-02-24,Danida,34,Da,38.852669,-6.582841,Kwambwezi,0,...,never pay,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
14846,18990,1000.0,2011-03-21,Hiap,0,HIAP,37.451633,-5.350428,Bonde La Mkondoa,0,...,annually,salty,salty,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
14847,28749,0.0,2013-03-04,NaN,1476,NaN,34.739804,-4.585587,Bwawani,0,...,never pay,soft,good,insufficient,insufficient,dam,dam,surface,communal standpipe,communal standpipe
14848,33492,0.0,2013-02-18,Germany,998,DWE,35.432732,-10.584159,Kwa John,0,...,never pay,soft,good,insufficient,insufficient,river,river/lake,surface,communal standpipe,communal standpipe


In [69]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier

bagged_tree =  BaggingClassifier(DecisionTreeClassifier(criterion='gini', max_depth=5), 
                                 n_estimators=100)
bagged_tree.fit(X_train,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                  n_estimators=100)

In [71]:
bagged_tree.score(X_test, y_test)

0.7025589225589226

In [72]:
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings("ignore")

In [77]:
model = XGBClassifier()
param_grid = {
    "learning_rate": [.05, 0.1, .15, .2],
    'max_depth': [3,4,5,6,7,8],
    'min_child_weight': [1,3,5,7],
    'gamma': [ 0.0, 0.1, 0.2, 0.3, 0.4],
    'colsample_bytree': [.3,.4,.5,.6,.7],
}
grid_clf = RandomizedSearchCV(model, param_grid, scoring='accuracy', cv=None, n_jobs=1)

scaler = StandardScaler()
X_train_transformed = scaler.fit_transform(X_train)
grid_clf.fit(X_train_transformed, y_train)


[12:02:42] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:03:05] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:03:29] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:03:52] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the object

[12:12:28] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:12:47] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:13:23] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:13:59] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the object

RandomizedSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100, n_job...
                                           random_state=None, reg_alpha=None,
                                           reg_lambda=None,
                                           scale_pos_weight=No

In [81]:
X_test_transformed = scaler.transform(X_test)
grid_clf.score(X_test_transformed, y_test)

0.7847138047138047

In [156]:
training_preds = model.predict(X_train_transformed)
training_accuracy = accuracy_score(y_train, training_preds)
testing_preds = model.predict(X_test_transformed)
testing_accuracy = accuracy_score(y_test, testing_preds)
testing_accuracy

0.7822222222222223

In [157]:
model.score(X_test_transformed, y_test)
testing_preds = model.predict(X_test_transformed)
testing_accuracy = accuracy_score(y_test, testing_preds)
testing_accuracy

0.7822222222222223

In [1]:
print("hi")

hi


In [188]:
xgb_grid_search.best_score_

AttributeError: 'GridSearchCV' object has no attribute 'best_score_'